In [63]:
import os
import pandas as pd

RAW_DIR = r"C:\Users\12432\Desktop\archive"
PROCESSED_DIR = "./data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

orders = pd.read_csv(
    RAW_DIR + r"\olist_orders_dataset.csv",
    low_memory=False,
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ],
)
customers = pd.read_csv(RAW_DIR + r"\olist_customers_dataset.csv", low_memory=False)
items     = pd.read_csv(RAW_DIR + r"\olist_order_items_dataset.csv",
                        low_memory=False, parse_dates=["shipping_limit_date"])
payments  = pd.read_csv(RAW_DIR + r"\olist_order_payments_dataset.csv", low_memory=False)
reviews   = pd.read_csv(RAW_DIR + r"\olist_order_reviews_dataset.csv",
                        low_memory=False, parse_dates=["review_creation_date", "review_answer_timestamp"])
products  = pd.read_csv(RAW_DIR + r"\olist_products_dataset.csv", low_memory=False)
sellers   = pd.read_csv(RAW_DIR + r"\olist_sellers_dataset.csv", low_memory=False)
geo       = pd.read_csv(RAW_DIR + r"\olist_geolocation_dataset.csv", low_memory=False)
catmap    = pd.read_csv(RAW_DIR + r"\product_category_name_translation.csv", low_memory=False)



In [65]:
orders["actual_delivery_days"] = (
    orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]
).dt.days

orders["delivery_delay_days"] = (
    orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"]
).dt.days

orders["is_late"] = orders["delivery_delay_days"] > 0

orders[["actual_delivery_days","delivery_delay_days","is_late"]].describe(include="all")


,actual_delivery_days,delivery_delay_days,is_late
count,96476.000000,96476.000000,99441
unique,NaN,NaN,2
top,NaN,NaN,False
freq,NaN,NaN,92906
mean,12.094086,-11.876881,NaN
std,9.551746,10.183854,NaN
min,0.000000,-147.000000,NaN
25%,6.000000,-17.000000,NaN
50%,10.000000,-12.000000,NaN
75%,15.000000,-7.000000,NaN


In [56]:
orders.to_parquet(PROCESSED_DIR + "/orders.parquet", index=False)
customers.to_parquet(PROCESSED_DIR + "/customers.parquet", index=False)
items.to_parquet(PROCESSED_DIR + "/order_items.parquet", index=False)
payments.to_parquet(PROCESSED_DIR + "/order_payments.parquet", index=False)
reviews.to_parquet(PROCESSED_DIR + "/order_reviews.parquet", index=False)
products.to_parquet(PROCESSED_DIR + "/products.parquet", index=False)
sellers.to_parquet(PROCESSED_DIR + "/sellers.parquet", index=False)
geo.to_parquet(PROCESSED_DIR + "/geolocation.parquet", index=False)
catmap.to_parquet(PROCESSED_DIR + "/category_translation.parquet", index=False)


In [57]:
orders_chk = pd.read_parquet(PROCESSED_DIR + "/orders.parquet")
items_chk  = pd.read_parquet(PROCESSED_DIR + "/order_items.parquet")

print("orders_chk:", orders_chk.shape, "items_chk:", items_chk.shape)
orders_chk[["order_purchase_timestamp","order_delivered_customer_date","actual_delivery_days"]].head(3)


orders_chk: (99441, 11) items_chk: (112650, 7)


,order_purchase_timestamp,order_delivered_customer_date,actual_delivery_days
0,2017-10-02 10:56:33,2017-10-10 21:25:13,8.0
1,2018-07-24 20:41:37,2018-08-07 15:27:45,13.0
2,2018-08-08 08:38:49,2018-08-17 18:06:29,9.0
